In [1]:
import os
import math
from glob import glob
import random
import itertools
import numpy as np
import pandas as pd
from scipy import stats
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models, layers


Button

In [2]:
test_type = 'opt'
top1 = True
base_path = f'[USER_PATH]\\ArtificialVision'

num_class = 16
if test_type == 'opt':
  sel_ppl = list(range(300, 309)) + list(range(400, 408)) + [611]
  hid_num = 3
elif test_type == 'elec': # check
  sel_ppl = [499, 500] + list(range(502, 509)) + list(range(601, 607)) + list(range(608, 611))
  hid_num = 4
if top1:
  acc_type = 'top1'
else:
  acc_type = 'top2'

quesdata_files = glob(f'{base_path}\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\*')
humdata_path = glob(f'{base_path}\\data\\Human_Expert\\211202')[0]
humsub_path = glob(f'{base_path}\\data\\210827_ANNA_Removing_uncontaminated_data.csv')[0]
answer_path = glob(f'{base_path}\\data\\211105_QAs_for_Set0_CNN_SVC_4classes_partial.csv')[0]
testdata_path = glob(f'{base_path}\\data\\sample_for_dev_test_{test_type}')[0]
fig_path = glob(f'{base_path}\\figures')[0]
checkpoint_path = glob(f'{base_path}\\checkpoints')[0]

Human Data

In [3]:
human_df = pd.DataFrame()
n = 9
for i in range(1, 80*n+1, 80):
    try:
      j = i+79
      temp_df = pd.read_csv(os.path.join(humdata_path, f'main_test({i}_{j}).csv'))
      if i == 1:
        pass
      else:
        temp_df = temp_df.rename(columns = {'유저식별아이디':'useless', 'MC구분':'useless', '성별':'useless', '나이':'useless', '학력':'useless'})
      human_df = pd.concat([human_df, temp_df], axis=1)
    except:
      print(i)

human_df = human_df[human_df['유저식별아이디'].isin(sel_ppl)]
orig_human_df = human_df
human_df = human_df.fillna(0)

sel_col = []
for j in range(1, 80*n+1):
  if top1:
    temp_str = f'선택_A_{j}'
    sel_col.append(temp_str)
  else:
    temp_str1 = f'선택_A_{j}'
    temp_str2 = f'선택_B_{j}'
    sel_col.append(temp_str1)
    sel_col.append(temp_str2)

human_df = human_df[sel_col]
human_df.index = sel_ppl 

if top1:
  human_df.columns = list(range(80*n))
else:
  human_df.columns = list(range(2*80*n))

human_df = human_df.T

answer_df = pd.read_csv(answer_path)

for (i, img) in enumerate(answer_df['Answer'].values):
    per, _, _, _ = img.split('_')
    if top1:
      k = i
    else:
      k = i*2
    for j in range(human_df.shape[1]):
      if top1:
        cond = int(per) == int(human_df.iloc[k, j])
      else:
        cond = (int(per) == int(human_df.iloc[k, j])) or (int(per) == int(human_df.iloc[k+1, j]))
      if cond:
          human_df.iloc[k, j] = 1
      else:
          human_df.iloc[k, j] = 0

if not top1:
  human_df = human_df[::2]
human_df.index = list(answer_df['Answer'].values)
human_df

,300,301,302,303,304,305,306,307,308,400,401,402,403,404,405,406,407,611
19070311_16PIX_8GS_S001L1E03C7.jpg,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19080133_32PIX_4GS_S001L1E03C7.jpg,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
19090222_32PIX_8GS_S001L1E01C4.jpg,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
19090631_16PIX_2GS_S001L1E03C7.jpg,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
19072221_32PIX_2GS_S001L1E01C4.jpg,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19072221_16PIX_8GS_S001L1E01C7.jpg,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
19082131_16PIX_2GS_S001L1E01C7.jpg,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
19092521_32PIX_4GS_S001L1E01C7.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19081632_64PIX_2GS_S001L1E01C7.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


Answer Data

In [4]:
act_per_list, pix_list, gs_list, par_list = [], [], [], []
for answer in answer_df['Answer']:
    img, _ = answer.split('.jpg')
    act_per, pix, gs, par = img.split('_')
    
    act_per_list.append(act_per)
    pix_list.append(pix)
    gs_list.append(gs)
    par_list.append(par)

answer_df['act_per'] = act_per_list
answer_df['PIX'] = pix_list
answer_df['GS'] = gs_list
answer_df['par'] = par_list

answer_df = answer_df[:80*n]
answer_df = answer_df.T

orig_answer_df = answer_df
answer_df

,0,1,2,3,4,5,6,7,8,9,...,710,711,712,713,714,715,716,717,718,719
Answer,19070311_16PIX_8GS_S001L1E03C7.jpg,19080133_32PIX_4GS_S001L1E03C7.jpg,19090222_32PIX_8GS_S001L1E01C4.jpg,19090631_16PIX_2GS_S001L1E03C7.jpg,19072221_32PIX_2GS_S001L1E01C4.jpg,19072221_64PIX_4GS_S001L1E03C7.jpg,19081632_64PIX_2GS_S001L1E03C7.jpg,19092711_64PIX_8GS_S001L1E02C7.jpg,19070522_16PIX_4GS_S001L1E03C7.jpg,19081421_32PIX_2GS_S001L1E01C7.jpg,...,19090631_32PIX_8GS_S001L1E01C7.jpg,19070231_64PIX_4GS_S001L1E03C7.jpg,19090222_16PIX_4GS_S001L1E03C7.jpg,19080133_32PIX_8GS_S001L1E01C7.jpg,19081421_32PIX_2GS_S001L1E01C4.jpg,19072221_16PIX_8GS_S001L1E01C7.jpg,19082131_16PIX_2GS_S001L1E01C7.jpg,19092521_32PIX_4GS_S001L1E01C7.jpg,19081632_64PIX_2GS_S001L1E01C7.jpg,19071131_64PIX_8GS_S001L1E01C7.jpg
MC1,"[19090631, 19070311, 19082032, 19071821]","[19090631, 19090222, 19070522, 19080133]","[19072922, 19071821, 19090222, 19070231]","[19070522, 19090222, 19080133, 19090631]","[19070231, 19070311, 19092521, 19072221]","[19082131, 19072221, 19070311, 19070522]","[19082032, 19081632, 19071131, 19092711]","[19092711, 19070231, 19082032, 19070311]","[19092521, 19072922, 19070311, 19070522]","[19081421, 19071131, 19081632, 19072922]",...,"[19081632, 19080133, 19090631, 19092711]","[19070231, 19080133, 19072922, 19071821]","[19081632, 19092711, 19070231, 19090222]","[19072922, 19072221, 19080133, 19090631]","[19082032, 19072221, 19090631, 19081421]","[19070231, 19092521, 19072221, 19080133]","[19092711, 19090631, 19082131, 19092521]","[19092521, 19070231, 19082131, 19090631]","[19092521, 19081632, 19070231, 19070522]","[19081421, 19090222, 19071131, 19082032]"
MC1_hit_rate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.666667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
MC2,"[19070311, 19071131, 19082131, 19070522]","[19081632, 19080133, 19071821, 19082032]","[19090222, 19081421, 19070311, 19070522]","[19090631, 19070231, 19090222, 19072221]","[19072922, 19071821, 19072221, 19081632]","[19092711, 19072922, 19072221, 19090631]","[19072922, 19082131, 19081632, 19080133]","[19071821, 19082131, 19092711, 19080133]","[19081421, 19070522, 19072221, 19081632]","[19072221, 19080133, 19070522, 19081421]",...,"[19071821, 19070231, 19092521, 19090631]","[19070231, 19081421, 19092521, 19072221]","[19090631, 19090222, 19071131, 19092711]","[19081632, 19072221, 19092521, 19080133]","[19081421, 19071131, 19072922, 19092711]","[19081421, 19092711, 19072221, 19071131]","[19090222, 19070231, 19082131, 19080133]","[19071821, 19081632, 19092521, 19072922]","[19082032, 19081632, 19071821, 19090222]","[19081632, 19082032, 19071131, 19092711]"
MC2_hit_rate,1.0,1.0,1.0,1.0,1.0,1.0,0.888889,1.0,1.0,0.111111,...,0.0,1.0,1.0,1.0,0.888889,1.0,0.888889,0.888889,1.0,1.0
MC3,"[19070311, 19092711, 19082032, 19090222]","[19080133, 19081421, 19071131, 19070231]","[19090222, 19071821, 19082032, 19082131]","[19070311, 19071131, 19090631, 19080133]","[19070522, 19092521, 19072922, 19072221]","[19071821, 19090222, 19090631, 19072221]","[19092711, 19070311, 19081632, 19071821]","[19092711, 19082032, 19090222, 19080133]","[19071131, 19082032, 19070231, 19070522]","[19082032, 19090631, 19081421, 19092521]",...,"[19080133, 19081421, 19090631, 19070231]","[19071821, 19070231, 19082131, 19092711]","[19090222, 19071821, 19090631, 19071131]","[19072221, 19080133, 19071131, 19070231]","[19081421, 19071821, 19070522, 19072922]","[19090631, 19082032, 19072221, 19071821]","[19081632, 19082131, 19092521, 19092711]","[19081632, 19070231, 19092521, 19081421]","[19070522, 19071131, 19081632, 19070311]","[19072922, 19071821, 19070231, 19071131]"
MC3_hit_rate,0.111111,0.0,0.888889,0.222222,0.0,0.888889,0.0,0.666667,0.111111,0.0,...,0.0,1.0,0.444444,0.333333,0.0,0.333333,0.0,0.0,0.222222,0.888889
act_per,19070311,19080133,19090222,19090631,19072221,19072221,19081632,19092711,19070522,19081421,...,19090631,19070231,19090222,19080133,19081421,19072221,19082131,19092521,19081632,19071131
PIX,16PIX,32PIX,32PIX,16PIX,32PIX,64PIX,64PIX,64PIX,

Machine Data

In [5]:
model_type1 = ''
model_type2 = 'CNN_SVC'
seed_list = [22, 77, 2, 100, 81, 42, 7, 1, 55, 50] 
pix_order_list = ['16PIX', '24PIX', '32PIX', '64PIX', '128PIX']
gs_order_list = ['2GS', '4GS', '6GS', '8GS', '16GS']

df = pd.read_csv(humsub_path)
l = list(range(df.shape[0]))
n = 16
random.seed(22)
set_1 = random.sample(l, n)
sets = [set_1]
face_labels = [19081632,
               19080133,
               19092711,
               19070311,
               19090631,
               19092521,
               19071821,
               19081421,
               19090222,
               19082032,
               19082131,
               19070231, 
               19070522,
               19071131,
               19072221,
               19072922]
face_dict = {i:face_labels[i] for i in range(len(face_labels))}


model_type = model_type1 + model_type2
input_folder = [df.iloc[i, 0] for i in sets[0]] 
assert len(input_folder) == 16
com_obj = itertools.combinations(input_folder, 16)
com_list = list(com_obj)

mac_df = pd.DataFrame()
for i in range(len(quesdata_files)):
    data_path = quesdata_files[i]
    seed = int(os.path.basename(data_path).split('d')[-1])

    for n in range(len(os.listdir(data_path))):
        print(seed, n)

        preprocessed_data_path =  os.path.join(data_path, f'comb{n}') 
        high_analysis_path = os.path.join(preprocessed_data_path, f'High_Analysis_{test_type}')
        
        add_high_df = pd.read_csv(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv'))
        add_high_df['Hit Rate'] = add_high_df['correctness'].replace(['correct', 'wrong'], [1, 0]) 
        add_high_df['actual_person'] = add_high_df['actual_person'].astype(int)

        if not top1:
            d = {i:[] for i in range(16)}
        
        img_list, hyperpar_list, par_list = [], [], []
        for i in range(add_high_df.shape[0]):
            file_name = add_high_df['file_name'][i].split('.')[0]
            pix, gs, par = file_name.split('_')
            per = add_high_df['actual_person'][i]
            img = str(per) + '_' + str(pix) + '_' + str(gs) + '_' + str(par) + '.jpg'
            img_list.append(img)
            hyperpar_list.append(str(pix) + '_' + str(gs))
            par_list.append(str(par))

            if not top1:
                temp_str = add_high_df['pred_prob_vector'].iloc[i]
                temp_split = temp_str.split('[')
                temp_split = temp_split[1].split(']')
                temp_split = temp_split[0].split(' ')
                temp_split = [comp for comp in temp_split if comp != '']
        
                for (p, num) in enumerate(temp_split):
                    num = float(num)
                    d[p].append(num)
                val = list(d.values())
                val.sort()
                max2 = val[-2]
                s = {k for k in d if d[k]==max2}
                if int(per) == int(face_dict[next(iter(s))]):
                    add_high_df['Hit Rate'][i] = 1
            
        add_high_df['img'] = img_list
        add_high_df['hyperpar'] = hyperpar_list
        add_high_df['par'] = par_list

        add_high_df = add_high_df[['img', 'hyperpar', 'par', 'Hit Rate']]
    
    add_high_df['Seed'] = [seed] * add_high_df.shape[0]
    
    mac_df = pd.concat([mac_df, add_high_df], axis=0)

new_hyperpar_name_list = ['16PIX_2GS', '16PIX_4GS', '16PIX_6GS', '16PIX_8GS', '16PIX_16GS',
                          '24PIX_2GS', '24PIX_4GS', '24PIX_6GS', '24PIX_8GS', '24PIX_16GS',
                          '32PIX_2GS', '32PIX_4GS', '32PIX_6GS', '32PIX_8GS', '32PIX_16GS',
                          '64PIX_2GS', '64PIX_4GS', '64PIX_6GS', '64PIX_8GS', '64PIX_16GS',
                          '128PIX_2GS', '128PIX_4GS', '128PIX_6GS', '128PIX_8GS', '128PIX_16GS']
par_list = ['S001L1E01C4', 'S001L1E01C7', 'S001L1E01C10',
            'S001L1E02C7',
            'S001L1E03C7']
mac_df = mac_df[mac_df['hyperpar'].isin(new_hyperpar_name_list)]
mac_df = mac_df[mac_df['par'].isin(par_list)]

mac_df = mac_df.pivot(index='Seed', columns='img', values='Hit Rate')

mac_df_copy = mac_df.copy()

mac_df_T = mac_df_copy.T

hyper_par_list = []
for img in mac_df_T.index:
    _, pix, gs, _ = img.split('_')
    hyper_par_list.append(f'{pix}_{gs}')
mac_df_T['hyperpar'] = hyper_par_list
mac_df_T_gp = mac_df_T.groupby('hyperpar').mean()

mac_df_T_gp.mean(axis=1)

mac_df_T_gp

1 0
100 0
2 0
22 0
42 0
50 0
55 0
7 0
77 0
81 0


Seed,1,2,7,22,42,50,55,77,81,100
hyperpar,,,,,,,,,,
128PIX_16GS,0.9125,0.8875,0.8875,0.8750,0.8875,0.8625,0.8875,0.8250,0.8625,0.8750
128PIX_2GS,0.8125,0.7500,0.7250,0.6250,0.6875,0.6750,0.7250,0.6625,0.6125,0.6750
128PIX_4GS,0.7875,0.8750,0.7750,0.7750,0.7875,0.8000,0.8375,0.7875,0.8375,0.8125
128PIX_6GS,0.8500,0.9125,0.8500,0.8000,0.8250,0.8000,0.8625,0.7875,0.8500,0.8000
128PIX_8GS,0.8875,0.8875,0.8750,0.8375,0.8750,0.8625,0.8625,0.8125,0.8500,0.8625
16PIX_16GS,0.4625,0.4500,0.3500,0.4625,0.5125,0.4625,0.5000,0.5125,0.4750,0.4625
16PIX_2GS,0.2375,0.2125,0.1500,0.2000,0.2875,0.1500,0.3000,0.2250,0.2250,0.2250
16PIX_4GS,0.3375,0.3750,0.2875,0.3250,0.4125,0.3750,0.4250,0.3625,0.4500,0.4125
16PIX_6GS,0.3625,0.4500,0.3625,0.4000,0.4500,0.4250,0.4625,0.4125,0.5125,0.4250


Humans & Machines

In [6]:
merged_df = pd.concat([human_df.T, mac_df], join='outer')
merged_df = merged_df[merged_df.index != 'img']

merged_df = merged_df.T.astype(float)

temp_index = []
for file_name in merged_df.index:
    split = file_name.split('.')
    face, pix, gs, par = split[0].split('_')
    temp_index.append(f'{pix}_{gs}') 

merged_df['idx'] = temp_index
temp_acc_df = merged_df.groupby('idx').mean()

temp_acc_df.columns = temp_acc_df.columns.astype(str)

# People
sel_ppl_str = list(str(per) for per in sel_ppl)
ppl_df = temp_acc_df[sel_ppl_str]
ppl_df_avg = ppl_df.mean(axis=1)
ppl_df_std = ppl_df.std(axis=1)

# Machines
sel_mac_str = list(str(per) for per in seed_list)
mod_df = temp_acc_df[sel_mac_str]
mod_df_avg = mod_df.mean(axis=1)
mod_df_std = mod_df.std(axis=1)

mer_avg = pd.concat([ppl_df_avg, ppl_df_std, mod_df_avg, mod_df_std], axis=1)
mer_avg.columns = ['HumansAvg', 'HumansStd', 'MachinesAvg', 'MachinesStd']
mer_avg = mer_avg.reset_index()
mer_avg

,idx,HumansAvg,HumansStd,MachinesAvg,MachinesStd
0,128PIX_16GS,NaN,NaN,0.87625,0.023162
1,128PIX_2GS,NaN,NaN,0.69500,0.059861
2,128PIX_4GS,NaN,NaN,0.80750,0.032914
3,128PIX_6GS,NaN,NaN,0.83375,0.038663
4,128PIX_8GS,NaN,NaN,0.86125,0.023162
5,16PIX_16GS,NaN,NaN,0.46500,0.046323
6,16PIX_2GS,0.368056,0.093858,0.22125,0.048965
7,16PIX_4GS,0.381250,0.067417,0.37625,0.050156
8,16PIX_6GS,NaN,NaN,0.42625,0.045814
9,16PIX_8GS,0.465972,0.064284,0.47125,0.056535


1. Results of Linear Model

In [24]:
new_hyperpar_name_list = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
                          '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',
                          '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']

per_mer_avg = mer_avg[mer_avg['idx'].isin(new_hyperpar_name_list)]

class LinearReg():

    def __init__(self, new_x):
        self.new_x = new_x
    
    def equation_val(self, new_x):
        x, y = per_mer_avg['MachinesAvg'], per_mer_avg['HumansAvg']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        new_y = slope * new_x + intercept
        return new_y
    
    @staticmethod
    def equation():
        x, y = per_mer_avg['MachinesAvg'], per_mer_avg['HumansAvg']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        print(f'R: {r_value}, p-value: {p_value}')
        return slope, intercept
                          
par_mac_df_T_gp = mac_df_T_gp[np.logical_not(mac_df_T_gp.index.isin(new_hyperpar_name_list))]
df = par_mac_df_T_gp.mean(axis=1).to_frame()
df.columns = ['Hit Rate of Machines']
diction = {key: [] for key in df.index}
linear_eq = LinearReg(df['Hit Rate of Machines'].values)
slope, intercept = linear_eq.equation()
print(f'Slope: {slope}, intercept: {intercept}')
for i in range(df.shape[0]):
    new_y = linear_eq.equation_val(df['Hit Rate of Machines'].iloc[i])
    
    diction[df.index[i]].append(df['Hit Rate of Machines'].iloc[i])
    diction[df.index[i]].append(new_y)

new_df = pd.DataFrame.from_dict(diction).T
new_df.columns = ['Machines', 'Humans']
new_df

R: 0.9750868587329597, p-value: 7.84278719179682e-06
Slope: 0.9034811584473966, intercept: 0.10044387793507148


,Machines,Humans
128PIX_16GS,0.87625,0.892119
128PIX_2GS,0.69500,0.728363
128PIX_4GS,0.80750,0.830005
128PIX_6GS,0.83375,0.853721
128PIX_8GS,0.86125,0.878567
16PIX_16GS,0.46500,0.520563
16PIX_6GS,0.42625,0.485553
24PIX_16GS,0.63000,0.669637
24PIX_2GS,0.33000,0.398593
24PIX_4GS,0.58250,0.626722


In [25]:
mer_avg.to_csv(os.path.join(fig_path, f'real_linear_{acc_type}_{test_type}.csv'))
new_df.to_csv(os.path.join(fig_path, f'pred_linear_{acc_type}_{test_type}.csv'))

2. Results of Non-linear Model

In [7]:
fb_df = mac_df.T

par_list = []
for file_name in fb_df.index:
    _, pix, gs, par = file_name.split('_')
    par_list.append(f'{pix}_{gs}_{par}')
fb_df.index = par_list

unique_pars = list(set(fb_df.index.values))
df = pd.DataFrame()
for par in unique_pars:
    df[par] = fb_df.loc[[par], :].mean(axis=0)
old_df = df.mean(axis=0).to_frame()

temp_list1, temp_list2 = [], []
for ind in old_df.index:
    pix, gs, par = ind.split('_')
    temp_list1.append(f'{pix}_{gs}')
    temp_list2.append(par.split('.')[0])
old_df['hyperpar'] = temp_list1
old_df['par'] = temp_list2

hyperpar_list = ['16PIX_6GS', '16PIX_16GS',
                 '24PIX_2GS', '24PIX_4GS', '24PIX_6GS', '24PIX_8GS', '24PIX_16GS',
                 '32PIX_6GS', '32PIX_16GS',
                 '64PIX_6GS', '64PIX_16GS',
                 '128PIX_2GS', '128PIX_4GS', '128PIX_6GS', '128PIX_8GS', '128PIX_16GS']
par_list = ['S001L1E01C4', 'S001L1E01C7', 'S001L1E01C10',
            'S001L1E02C7',
            'S001L1E03C7']

test_df = old_df[old_df['hyperpar'].isin(hyperpar_list)]
test_df = old_df[old_df['par'].isin(par_list)]

test_df.columns = ['acc', 'hyperpar', 'par']
test_df = test_df.pivot(index='hyperpar', columns='par', values='acc')
test_df['corr'] = [0.7598] * test_df.shape[0]

test_df

C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\754810066.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[par] = fb_df.loc[[par], :].mean(axis=0)
C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\754810066.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[par] = fb_df.loc[[par], :].mean(axis=0)
C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\754810066.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

par,S001L1E01C10,S001L1E01C4,S001L1E01C7,S001L1E02C7,S001L1E03C7,corr
hyperpar,,,,,,
128PIX_16GS,0.94375,0.95000,0.79375,0.85000,0.84375,0.7598
128PIX_2GS,0.76875,0.73750,0.65625,0.63125,0.68125,0.7598
128PIX_4GS,0.91875,0.86250,0.75000,0.76250,0.74375,0.7598
128PIX_6GS,0.91875,0.90625,0.76875,0.79375,0.78125,0.7598
128PIX_8GS,0.93125,0.94375,0.79375,0.83125,0.80625,0.7598
16PIX_16GS,0.39375,0.46875,0.51250,0.48125,0.46875,0.7598
16PIX_2GS,0.17500,0.16250,0.33125,0.13750,0.30000,0.7598
16PIX_4GS,0.36250,0.31875,0.42500,0.36250,0.41250,0.7598
16PIX_6GS,0.38125,0.43125,0.41250,0.46875,0.43750,0.7598


In [8]:
class ANN():
    def __init__(self, x, checkpoint_path):
        self.x = x
        self.checkpoint_path = checkpoint_path
        
    def load_model(self):
        model = keras.models.load_model(self.checkpoint_path)
        return model
    
    def eval_mod(self, model):
        return model(self.x)

In [9]:
keras.models.load_model(os.path.join(checkpoint_path, f'Checkpoint_{hid_num}.h5'))

In [30]:
X_test = test_df.values

ann_inst = ANN(X_test, checkpoint_path=os.path.join(checkpoint_path, f'Checkpoint_{hid_num}.h5'))
model = ann_inst.load_model()
y_test = ann_inst.eval_mod(model)
pred_df = pd.DataFrame(y_test, columns=old_df.columns[:-1], index=old_df.index)
pred_df = pred_df.mean(axis=1).to_frame()
pred_df.columns = ['Human']
pred_df

,Human
hyperpar,
128PIX_16GS,0.830836
128PIX_2GS,0.674439
128PIX_4GS,0.770859
128PIX_6GS,0.798863
128PIX_8GS,0.826213
16PIX_16GS,0.482505
16PIX_6GS,0.422215
24PIX_16GS,0.624148
24PIX_2GS,0.401325


In [31]:
pred_df.to_csv(os.path.join(fig_path, f'pred_nonlinear_{acc_type}_{test_type}.csv'))

How did we generate the checkpoints?

In [10]:
cor_type = 'pear'

fb_df = merged_df.copy()

par_list = []
for file_name in fb_df.index:
    _, pix, gs, par = file_name.split('_')
    par_list.append(f'{pix}_{gs}_{par}')
fb_df.index = par_list

df = pd.DataFrame()
for par in unique_pars:
    df[par] = fb_df.loc[[par], :].mean(axis=0)
old_df = df.T

hyperpar_list = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS',
                '32PIX_2GS', '32PIX_4GS', '32PIX_8GS', 
                '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
par_list = ['S001L1E01C4', 'S001L1E01C7', 'S001L1E01C10',
            'S001L1E02C7',
            'S001L1E03C7']
temp_list1, temp_list2 = [], []
for ind in old_df.index:
    pix, gs, par = ind.split('_')
    temp_list1.append(f'{pix}_{gs}')
    temp_list2.append(par.split('.')[0])
old_df['hyperpar'] = temp_list1
old_df['par'] = temp_list2

train_df = pd.DataFrame()
for i in sel_ppl:
    for j in seed_list:
        for l, hyperpar in enumerate(hyperpar_list): 
            par_old_df = old_df[old_df['hyperpar'] == hyperpar]
            j = int(j)
            par_train_df = par_old_df[[i, j]]
            hum, mac = par_old_df[i], par_old_df[j] 
            
            if cor_type == 'pear':
                cor_val = stats.pearsonr(hum, mac) 

            elif cor_type == 'adj_pear':
                indexes = list(range(par_train_df.shape[0]))
                avg_l = []
                for p in range(10):
                    split_half_ran_ind1 = random.sample(indexes, par_train_df.shape[0]//2)
                    split_half_ran_ind2 = [ind for ind in indexes if ind not in split_half_ran_ind1][1:]
                    
                    hum_par1 = hum.to_frame().iloc[split_half_ran_ind1, :].squeeze()
                    hum_par2 = hum.to_frame().iloc[split_half_ran_ind2, :].squeeze()
                    mac_par1 = mac.to_frame().iloc[split_half_ran_ind1, :].squeeze()
                    mac_par2 = mac.to_frame().iloc[split_half_ran_ind2, :].squeeze()

                    num = (stats.pearsonr(hum_par1, mac_par1)[0] + stats.pearsonr(hum_par1, mac_par2)[0] + stats.pearsonr(hum_par2, mac_par1)[0] + stats.pearsonr(hum_par2, mac_par2)[0]) / 4
                    den = math.sqrt(abs(stats.pearsonr(hum_par1, hum_par2)[0] * stats.pearsonr(mac_par1, mac_par2)[0]))
                    cor_val = num / den

                    avg_l.append(cor_val)

                cor_val = np.mean(avg_l)

            temp_train_df = pd.concat([hum, mac], axis=0).to_frame().T 
            
            par_list = list(range(1, 6))
            temp_train_df.columns = [f'Human_{j}' for j in par_list] + [f'ANN_{i}' for i in par_list]
            temp_train_df['Avg of ANN'] = temp_train_df[[f'ANN_{i}' for i in par_list]].mean(axis=1)
            temp_train_df['Cor'] = cor_val[0]
            temp_train_df['Avg of Hum'] = temp_train_df[[f'Human_{i}' for i in par_list]].mean(axis=1)

            temp_train_df = temp_train_df[[f'ANN_{j}' for j in list(range(1, 6))] + ['Cor'] + [f'Human_{i}' for i in list(range(1, 6))]]

            train_df = pd.concat([train_df, temp_train_df], axis=0)

train_df

C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\3564928794.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[par] = fb_df.loc[[par], :].mean(axis=0)
C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\3564928794.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[par] = fb_df.loc[[par], :].mean(axis=0)
C:\Users\Na Min An\AppData\Local\Temp\ipykernel_7760\3564928794.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[par] = fb_df.loc[[par], :].mean(axis=0)
C:\Users\N

,ANN_1,ANN_2,ANN_3,ANN_4,ANN_5,Cor,Human_1,Human_2,Human_3,Human_4,Human_5
0,0.1250,0.1875,0.1875,0.1875,0.3125,-0.583874,0.5625,0.6250,0.3750,0.3125,0.3125
0,0.3750,0.3125,0.3750,0.2500,0.3125,-0.422577,0.4375,0.5000,0.5000,0.5000,0.5625
0,0.5000,0.5000,0.3750,0.4375,0.4375,-0.071429,0.5625,0.4375,0.5000,0.5625,0.5000
0,0.5625,0.6250,0.5625,0.4375,0.2500,0.474675,0.8125,0.8750,0.6250,0.5625,0.6875
0,0.6250,0.8125,0.8125,0.5625,0.5625,-0.788486,1.0000,0.6875,0.7500,0.8750,0.8750
...,...,...,...,...,...,...,...,...,...,...,...
0,0.6250,0.8125,0.6250,0.6250,0.7500,0.949071,0.5625,0.7500,0.5625,0.5625,0.6250
0,0.6875,0.7500,0.7500,0.8125,0.7500,-0.353553,0.6875,0.7500,0.7500,0.6250,0.6250
0,0.6875,0.4375,0.7500,0.6250,0.6875,-0.314800,0.7500,0.7500,0.6875,0.5625,0.5000
0,0.8750,0.7500,0.8750,0.6875,0.8125,0.202289,0.8750,0.8750,0.6875,0.6875,0.8125


In [11]:
X_train = train_df[[f'ANN_{j}' for j in par_list] + ['Cor']]
y_train = train_df[[f'Human_{i}' for i in par_list]]

model = models.Sequential()
model.add(keras.Input(shape=(6, )))
model.add(layers.Dense(hid_num, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(5, activation='relu'))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_absolute_error'])
os.chdir(checkpoint_path)
callback_list = [ModelCheckpoint(filepath=f'Checkpoint_{hid_num}_demo.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(X_train, y_train, batch_size=16, epochs=3000, validation_split=0.1, callbacks=callback_list)

Epoch 1/3000
92/92 [==============================] - 1s 3ms/step - loss: 0.3923 - mean_absolute_error: 0.5560 - val_loss: 0.3562 - val_mean_absolute_error: 0.5234
Epoch 2/3000
92/92 [==============================] - 0s 1ms/step - loss: 0.3503 - mean_absolute_error: 0.5079 - val_loss: 0.2475 - val_mean_absolute_error: 0.4091
Epoch 3/3000
92/92 [==============================] - 0s 1ms/step - loss: 0.2223 - mean_absolute_error: 0.3741 - val_loss: 0.1485 - val_mean_absolute_error: 0.2967
Epoch 4/3000
92/92 [==============================] - 0s 1ms/step - loss: 0.1493 - mean_absolute_error: 0.2909 - val_loss: 0.1207 - val_mean_absolute_error: 0.2565
Epoch 5/3000
92/92 [==============================] - 0s 1ms/step - loss: 0.1358 - mean_absolute_error: 0.2687 - val_loss: 0.1144 - val_mean_absolute_error: 0.2438
Epoch 6/3000
92/92 [==============================] - 0s 1ms/step - loss: 0.1315 - mean_absolute_error: 0.2603 - val_loss: 0.1117 - val_mean_absolute_error: 0.2378
Epoch 7/3000
92/